# Deep learning (visszacsatolt hálók)

A gyakorlaton egy egyszerű, deep learning-alapú nyelvi modellt valósítunk meg, amellyel magyar nyelvű szövegeket dolgozunk fel. Az implementáció alapját egy visszacsatolt neurális háló adja, amelyet a szövegben soron következő karakter predikciójára tanítunk (a korábban megfigyelt karaktereket, mint bemenetet felhasználva). A tanult modellt ezután szöveg generálására használjuk fel.

In [1]:
%pylab inline

import tensorflow as tf
from urllib.request import urlopen

Populating the interactive namespace from numpy and matplotlib


## 1. Bemeneti adatok előállítása

A karaktereket one-hot módon kódoljuk. Minden karaktert egy $\left\lbrace 0,1\right\rbrace^d$ vektorral reprezentálunk, ahol például

\begin{align}
\text{'a'} &= \left[1, 0, 0, \dots, 0 \right], \\
\text{'b'} &= \left[0, 1, 0, \dots, 0 \right], \\
& \vdots \\
\text{'Z'} &= \left[0, 0, 0, \dots, 1 \right], \\
\end{align}

A visszacsatolt hálózat bemenetére egy $\left(n,m,d\right)$ méretű, kimenetére egy $\left(n,d\right)$ méretű tenzor kerül, ahol 

- $n$ a tanítóminták száma
- Minden tanítómintában van a teljes szövegnek egy $m$ hosszú rész-szekvenciája, amelynek minden karakterét az előbb látott $d$-dimenziós vektorokkal kódoljuk (bemenet)
- Minden tanítómintában található egy $d$-dimenziós vektor (kimenet), amely az előbbi rész-szekvencia utáni első rákövetkező karaktert reprezentálja.

A tanítómintákat úgy generáljuk, hogy ezt az $m$ széles ablakot végigtoljuk a teljes szövegen. Összefoglalva, a háló a bemenet-kimenet összefüggés becslése során $m$ lépésnyit tekint vissza.

In [2]:
url = "https://www.mit.bme.hu/system/files/oktatas/targyak/10142/telep.txt" # nevek.txt, telep.txt
seq = urlopen(url).read().decode('utf8')

<b>1.1. feladat.</b> Hozzon létre egy kódoló és dekódoló dictionary-t a one-hot kódoláshoz (azaz előbbiben minden egyedi karakterhez szerepeljen egy egyedi $d$-nél kisebb szám, utóbbi legyen az előbbi inverze).

In [3]:
chars = set(seq)
d     = len(chars)

encoding = dict(zip(chars,range(d)))
decoding = dict(zip(range(d),chars))

<b>1.2. feladat.</b> Alkalmas $m$ választása mellett hozza létre háló bemenetére kerülő $(n,m,d)$ méretű tenzort, valamint a kimenetet reprezentáló $(n,d)$ méretű tenzort!

In [4]:
l = len(seq)
m = 32
w = 1
n = (l-m)//w

chars_in  = np.zeros((n,m,d),dtype=np.bool)
chars_out = np.zeros((n,d),dtype=np.bool)

for i in range(n):
    for j in range(m):
        chars_in[i,j,encoding[seq[i*w+j]]] = 1
    chars_out[i,encoding[seq[i*w+m]]] = 1

## 2. Visszacsatolt háló létrehozása

A hálózathoz tetszőleges architektúrát használhatunk, de ügyelnünk kell arra, hogy a hálózat kimenete kompatibilis legyen az előző lépésben létrehozott kimenettel, azaz a kimenet mérete legyen $d$ (ehhez például illeszthetünk egy teljesen összekötött réteget a visszacsatolt réteg után). Mivel lényegében többosztályos osztályozást végzünk, a veszteségfüggvényt és aktivációs függvényt is ennek megfelelően kell megválasztani. A hálózat létrehozásához itt talál segítséget:

https://keras.io/

(pl. Input, LSTM/GRU/SimpleRNN, Dense rétegek).

<b>2.1. feladat.</b> Hozzon létre a követelményeknek megfelelő neurális hálózatot, majd hozzon létre egy modellt (használja a <i>tf.keras.Model()</i> és  osztályt és ennek a <i>tf.keras.Model.compile()</i> metódusát).

In [6]:
inp   = tf.keras.layers.Input(shape=(m,d))
net   = tf.keras.layers.LSTM(128,dropout=0.5,recurrent_dropout=0.5)(inp)
net   = tf.keras.layers.Dense(d,activation=tf.nn.softmax)(net)
model = tf.keras.Model(inp,net)
model.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer=tf.keras.optimizers.RMSprop(lr=0.01))

<b>2.2. feladat.</b> Végezze el a tanítást (<i>tf.keras.Model.fit()</i> függvény).

In [8]:
model.fit(chars_in,chars_out,epochs=50,batch_size=256);

Epoch 1/50
29984/29984 [==============================] - 4s 141us/sample - loss: 2.7255
Epoch 2/50
29984/29984 [==============================] - 4s 142us/sample - loss: 2.5570
Epoch 3/50
29984/29984 [==============================] - 4s 143us/sample - loss: 2.4855
Epoch 4/50
29984/29984 [==============================] - 4s 142us/sample - loss: 2.4266
Epoch 5/50
29984/29984 [==============================] - 4s 141us/sample - loss: 2.3828
Epoch 6/50
29984/29984 [==============================] - 4s 148us/sample - loss: 2.3538
Epoch 7/50
29984/29984 [==============================] - 4s 144us/sample - loss: 2.3293
Epoch 8/50
29984/29984 [==============================] - 4s 144us/sample - loss: 2.3068
Epoch 9/50
29984/29984 [==============================] - 4s 140us/sample - loss: 2.2928
Epoch 10/50
29984/29984 [==============================] - 4s 144us/sample - loss: 2.2797
Epoch 11/50
29984/29984 [==============================] - 4s 148us/sample - loss: 2.2664
Epoch 12/50
29984/2

## 3. Szöveg generálása

A szöveg generálásához a következőképpen járhatunk el:

- Az első bemenetet létrehozhatjuk tetszőlegesen (pl. a szöveg egy részlete)
- Ezen bemenetet a tanult modellre ráadva megkapjuk a rákövetkező karakter eloszlását
- Az eloszlásból mintavételezzük a rákövetkező karaktert, amit ki is írunk
- A bemenetet shifteljük (az utolsó helyre az imént kapott karakter kerül, az első pedig kiesik)

Ezt tetszőleges ideig ismételgetjük.

<b>3.1. feladat.</b> Generáljon 200 karakternyi szöveget a tanult modell felhasználásával. Variálja az architektúrát, dokumentálja a tapasztaltakat.

In [9]:
pred = np.zeros((1,m,d),dtype=np.bool)
pred[0,-1,encoding['\n']] = 1
out = ""
for i in range(200):
    pr = np.random.choice(range(d),p=model.predict(pred)[0])
    pred = np.roll(pred,-1,1)
    pred[0,-1,:] = np.zeros(d)
    pred[0,-1,pr] = 1
    out += decoding[pr]
print(out)

Répényszentbarlán
Nemesszentgyörgyös
Tágonypötk
Oztverkék
Vémenke
Vejentpéthely
Szobageresztamjér
Sápos
Zalaszentmihály
Govácsvállós
Győrcgycsakás
Hejcs
Kegyházasú
Fommónos
Zilasziget
Nyírrács
Kispará


<b>3.2. szorgalmi.</b> Hozzon létre és tanítson egy bonyolultabb nyelvi modellt, amely hosszabb szövegek, pl. könyvek generálására is alkalmas.